IMDB才对

In [1]:
from fastai.text.all import *
import pandas as pd

数据准备，使用的完整数据集

In [2]:
path= untar_data(URLs.IMDB)

In [3]:
dls= TextDataLoaders.from_folder(
    path,
    valid='test',
    text_vocab_kwargs={'min_freq': 5},
    seq_len=72,
    bs=32,
)

查看数据样例

In [4]:
dls.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxrep 3 * xxup spoilers xxrep 3 * xxrep 3 * xxup spoilers xxrep 3 * xxmaj continued … \n\n xxmaj from here on in the whole movie collapses in on itself . xxmaj first we meet a rogue program with the indication we 're gon na get ghosts and vampires and werewolves and the like . xxmaj we get a guy with a retarded accent talking endless garbage , two ' ghosts ' that serve no real purpose and have no character what - so - ever and a bunch of henchmen . xxmaj someone 's told me they 're vampires ( straight out of xxmaj blade 2 ) , but they 're so undefined i did n't realise . \n\n xxmaj the funny accented guy with a ridiculous name suffers the same problem as the xxmaj oracle , only for far longer and far far worse .",neg
2,"xxbos xxunk ) is the developing world 's answer to xxmaj silence of the xxmaj lambs . xxmaj where ` silence ' terrorized our peace of mind , ` citizen ' exhausts and saddens us instead . xxmaj this dramatization of the xxmaj chikatilo case translates rather well , thanks to a xxmaj westernized friendship between two xxmaj rostov cops who become equals . \n\n citizenx may also argue against ( ! ) the death penalty far better than xxmaj kevin xxmaj spacey 's xxmaj the xxmaj life of xxmaj david xxmaj xxunk ) . \n\n xxmaj humans are xxmaj machiavellian mammals , under which lie limbic brains ( lizard - logic ) . xxmaj why did two kids , who knew better , stone to death a toddler they kidnapped ? xxmaj why do bloodthirsty women yell ` li - xxunk ' at acts of xxup obscene terrorism ?",pos


创建学习器

In [5]:
learn= text_classifier_learner(
    dls,
    AWD_LSTM,
    drop_mult=0.5,
    metrics=[accuracy]
)

模型训练

微调最后一层

In [6]:
learn.fine_tune(3, 2e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.457917,0.393939,0.824880,03:03


epoch,train_loss,valid_loss,accuracy,time
0,0.329366,0.276771,0.886400,04:55
1,0.306972,0.229845,0.909800,05:08
2,0.248103,0.219149,0.913520,05:51


解冻更多层

In [7]:
learn.freeze_to(-2)
learn.fit_one_cycle(3, slice(1e-4, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.273081,0.239906,0.900200,05:54
1,0.263483,0.221181,0.911600,06:04
2,0.229958,0.212113,0.917640,04:42


完全解冻

In [8]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-5, 1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.219277,0.206208,0.918360,07:26
1,0.220368,0.198308,0.923080,05:00
2,0.184368,0.204371,0.922400,04:48
3,0.153010,0.209185,0.924880,04:47
4,0.143884,0.209935,0.925040,04:47


模型评估

In [9]:
valid_loss, valid_acc= learn.validate()
print(f"Validation loss: {valid_loss:.4f}, Validation accuracy: {valid_acc:.4f}")

Validation loss: 0.2099, Validation accuracy: 0.9250


保存模型

In [10]:
learn.export('imdb_classifier.pkl')
# 加载时使用：load_learner('imdb_classifier.pkl')

预测

In [23]:
sample_review = "I loved this movie, it was great!"
pred = learn.predict(sample_review)
print(f"预测结果：{pred[0]}，置信度：{pred[2][1].item():.4f}")

预测结果：pos，置信度：1.0000


In [24]:
# 负面评价测试
negative_review = "it was a terrible movie, I hated it!"
pred2= learn.predict(negative_review)
print(f"预测结果：{pred2[0]}，置信度：{pred2[2][1].item():.4f}")

预测结果：neg，置信度：0.0008


In [30]:
negative_review2 = "中文数据集没有作用，我讨厌它！"
pred3= learn.predict(negative_review2)
print(f"预测结果：{pred3[0]}，置信度：{pred3[2][1].item():.4f}")

预测结果：pos，置信度：0.9924


很明显负面评价不对，不知道哪里出了问题(找到了，是英语数据集)

可以存在的改进地方

seq_len可以增加，以处理长影评
可以尝试吧transformer替换AWM_LSTM
